In [4]:
import os
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error 
from numpy import math
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
import matplotlib.pyplot as plt


In [15]:
# loading data
d = os.path.dirname(os.getcwd())
path = d+"\\data\\processed\\processed_data.csv"
tripdata = pd.read_csv(path)
tripdata.head()

,Unnamed: 0,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_date,pickup_weekday,pickup_month,pickup_year,...,hourbins,coordinates,mercator,mercator_x,mercator_y,direction,delta_longitude,delta_latitude,peak_hours,manhattan
0,104406,-73.954347,40.763839,-73.950884,40.771308,1.0,2013-09-22,6,9.0,2013.0,...,"(0, 5]","(40.763839, -73.954347)","(-8232560.24998906, 4977570.128221102)",-8.232560e+06,4.977570e+06,359.537472,0.003463,0.007469,1,0.010932
1,155552,-73.977028,40.743278,-73.986602,40.728143,1.0,2011-03-25,4,3.0,2011.0,...,"(5, 11]","(40.743278, -73.977028)","(-8235085.087359742, 4974548.654648803)",-8.235085e+06,4.974549e+06,179.371431,0.009574,0.015135,0,0.024709
2,75475,-73.983017,40.781655,-73.976823,40.788423,3.0,2009-01-07,2,1.0,2009.0,...,"(0, 5]","(40.781655, -73.983017)","(-8235751.779790102, 4980188.975376161)",-8.235752e+06,4.980189e+06,359.086132,0.006194,0.006768,0,0.012962
3,142151,-73.982161,40.757050,-73.959005,40.783415,1.0,2009-06-12,4,6.0,2009.0,...,"(5, 11]","(40.75705, -73.982161)","(-8235656.490305984, 4976572.369906419)",-8.235656e+06,4.976572e+06,359.123122,0.023156,0.026365,0,0.049521
4,109426,-73.958803,40.815331,-73.961285,40.811988,1.0,2011-03-22,1,3.0,2011.0,...,"(0, 5]","(40.815331, -73.958803)","(-8233056.289640035, 4985141.068750157)",-8.233056e+06,4.985141e+06,179.257899,0.002482,0.003343,0,0.005825


In [16]:
tripdata.columns

Index(['Unnamed: 0', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'passenger_count',
       'pickup_date', 'pickup_weekday', 'pickup_month', 'pickup_year',
       'pickup_hour', 'fare_amount', 'farebins', 'distance',
       'fare_amount_per_km', 'hourbins', 'coordinates', 'mercator',
       'mercator_x', 'mercator_y', 'direction', 'delta_longitude',
       'delta_latitude', 'peak_hours', 'manhattan'],
      dtype='object')

In [17]:
train = tripdata[['passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude','pickup_weekday', 'pickup_hour','pickup_month','pickup_year','delta_longitude', 'delta_latitude', 'distance','direction','fare_amount']]


In [18]:
train.head()

,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,pickup_weekday,pickup_hour,pickup_month,pickup_year,delta_longitude,delta_latitude,distance,direction,fare_amount
0,1.0,-73.954347,40.763839,-73.950884,40.771308,6,19.0,9.0,2013.0,0.003463,0.007469,0.880232,359.537472,5.0
1,1.0,-73.977028,40.743278,-73.986602,40.728143,4,17.0,3.0,2011.0,0.009574,0.015135,1.866275,179.371431,5.3
2,3.0,-73.983017,40.781655,-73.976823,40.788423,2,17.0,1.0,2009.0,0.006194,0.006768,0.915594,359.086132,4.1
3,1.0,-73.982161,40.757050,-73.959005,40.783415,4,15.0,6.0,2009.0,0.023156,0.026365,3.520959,359.123122,8.9
4,1.0,-73.958803,40.815331,-73.961285,40.811988,1,8.0,3.0,2011.0,0.002482,0.003343,0.426391,179.257899,3.3


In [19]:
d = os.path.dirname(os.getcwd())
path = d+"\\data\\processed\\test_data.csv"
testdata = pd.read_csv(path)
testdata.head()

,Unnamed: 0,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,passenger_count,pickup_date,pickup_weekday,pickup_month,pickup_year,pickup_hour,fare_amount,distance,direction,delta_longitude,delta_latitude,peak_hours,manhattan
0,54458,-73.961287,40.771617,-73.974797,40.761457,1,2009-11-12,3,11,2009,19,7.7,1.603380,178.674663,0.013510,0.010160,1,0.023670
1,193387,-73.937060,40.854562,-73.984868,40.773842,2,2013-02-17,6,2,2013,13,24.0,9.836146,179.408433,0.047808,0.080720,0,0.128528
2,57401,-73.978800,40.762280,-73.973973,40.762740,1,2011-12-24,5,12,2011,14,4.1,0.409743,349.652379,0.004827,0.000460,0,0.005287
3,115324,-73.958191,40.760422,-73.861557,40.865349,1,2015-02-09,0,2,2015,12,34.0,14.221987,359.077704,0.096634,0.104927,0,0.201561
4,71684,-73.951845,40.778344,-73.958517,40.783291,1,2009-11-03,1,11,2009,15,4.5,0.786243,1.346232,0.006672,0.004947,0,0.011619


In [20]:
train.columns

Index(['passenger_count', 'pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'pickup_weekday',
       'pickup_hour', 'pickup_month', 'pickup_year', 'delta_longitude',
       'delta_latitude', 'distance', 'direction', 'fare_amount'],
      dtype='object')

In [21]:
test= testdata[['passenger_count','pickup_longitude', 'pickup_latitude',
       'dropoff_longitude', 'dropoff_latitude', 'pickup_weekday', 'pickup_hour',
       'pickup_month', 'pickup_year', 'delta_longitude', 'delta_latitude',
       'distance', 'direction','fare_amount']] 
test.head()

,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,pickup_weekday,pickup_hour,pickup_month,pickup_year,delta_longitude,delta_latitude,distance,direction,fare_amount
0,1,-73.961287,40.771617,-73.974797,40.761457,3,19,11,2009,0.013510,0.010160,1.603380,178.674663,7.7
1,2,-73.937060,40.854562,-73.984868,40.773842,6,13,2,2013,0.047808,0.080720,9.836146,179.408433,24.0
2,1,-73.978800,40.762280,-73.973973,40.762740,5,14,12,2011,0.004827,0.000460,0.409743,349.652379,4.1
3,1,-73.958191,40.760422,-73.861557,40.865349,0,12,2,2015,0.096634,0.104927,14.221987,359.077704,34.0
4,1,-73.951845,40.778344,-73.958517,40.783291,1,15,11,2009,0.006672,0.004947,0.786243,1.346232,4.5


In [2]:
#defining function to print metrics for eval'passenger_couuation of models 
#independent_variables = [ 'pickup_longitude', 'pickup_latitude','dropoff_longitude', 'dropoff_latitude','pickup_hour', 'pickup_year', 'delta_longitude', 'delta_latitude','distance','direction']
       
       
independent_variables = [ 'pickup_longitude', 'pickup_latitude','dropoff_longitude', 'dropoff_latitude','passenger_count','distance','direction','pickup_hour']
RMSE_list = []


def print_metrics(actual, predicted,independent_variables):
  
  print('RMSE is {}'.format(math.sqrt(mean_squared_error(actual, predicted))))
  RMSE_list.append(math.sqrt(mean_squared_error(actual, predicted)))
 

In [3]:
import numpy as np
avg_fare=round(np.mean(train['fare_amount']),2)
baseline_pred=np.repeat(avg_fare,test['fare_amount'].shape[0])
baseline_rmse=np.sqrt(mean_squared_error(baseline_pred, test['fare_amount']))
print("Baseline RMSE of Validation data :",baseline_rmse)

NameError: name 'train' is not defined

In [ ]:
from sklearn.tree import DecisionTreeRegressor
dtregressor = DecisionTreeRegressor(random_state=0)
dtregressor.fit(train[independent_variables],train['fare_amount'])
y_test_predicted = dtregressor.predict(test[independent_variables])
y_train_pred = dtregressor.predict(train[independent_variables])

                    

In [12]:
print("Evaluation metrics for training data")
print("--------------------------------------")
print_metrics(train['fare_amount'],y_train_pred,independent_variables)
print("--------------------------------------")
print("Evaluation metrics for test data")
print("--------------------------------------")
print_metrics(test['fare_amount'], y_test_predicted,independent_variables)

Evaluation metrics for training data
--------------------------------------
RMSE is 3.60757014549187e-16
--------------------------------------
Evaluation metrics for test data
--------------------------------------
RMSE is 6.696367834878846


In [13]:
parameters={"splitter":["best","random"],
            "min_samples_leaf":[1,2,3,4,5],
           "min_samples_split":[2,3,4,5,6],
         
          }
from sklearn.model_selection import GridSearchCV
dtregressor = DecisionTreeRegressor(random_state=0)
tuning_model=GridSearchCV(dtregressor,param_grid=parameters,cv=3,verbose=3)
tuning_model.fit(train[independent_variables],train['fare_amount'])
y_test_predicted = tuning_model.predict(test[independent_variables])
y_train_pred = tuning_model.predict(train[independent_variables])

Fitting 3 folds for each of 50 candidates, totalling 150 fits
[CV 1/3] END min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.9s
[CV 2/3] END min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.8s
[CV 3/3] END min_samples_leaf=1, min_samples_split=2, splitter=best; total time=   0.8s
[CV 1/3] END min_samples_leaf=1, min_samples_split=2, splitter=random; total time=   0.1s
[CV 2/3] END min_samples_leaf=1, min_samples_split=2, splitter=random; total time=   0.1s
[CV 3/3] END min_samples_leaf=1, min_samples_split=2, splitter=random; total time=   0.1s
[CV 1/3] END min_samples_leaf=1, min_samples_split=3, splitter=best; total time=   0.8s
[CV 2/3] END min_samples_leaf=1, min_samples_split=3, splitter=best; total time=   0.8s
[CV 3/3] END min_samples_leaf=1, min_samples_split=3, splitter=best; total time=   0.8s
[CV 1/3] END min_samples_leaf=1, min_samples_split=3, splitter=random; total time=   0.1s
[CV 2/3] END min_samples_leaf=1, min_samples_split

KeyboardInterrupt: 

In [ ]:
tuning_model.best_params_

{'min_samples_leaf': 5, 'min_samples_split': 2, 'splitter': 'random'}

In [ ]:
print("Evaluation metrics for training data")
print("--------------------------------------")
print_metrics(train['fare_amount'],y_train_pred,independent_variables)
print("--------------------------------------")
print("Evaluation metrics for test data")
print("--------------------------------------")
print_metrics(test['fare_amount'], y_test_predicted,independent_variables)

Evaluation metrics for training data
--------------------------------------
RMSE is 3.0979973193998167
--------------------------------------
Evaluation metrics for test data
--------------------------------------
RMSE is 5.751172652206356


In [29]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(random_state=0)
regr.fit(train[independent_variables],train['fare_amount'])

RandomForestRegressor(random_state=0)

In [31]:
y_test_predicted = regr.predict(test[independent_variables])
y_train_pred = regr.predict(train[independent_variables])
print("Evaluation metrics for training data")
print("--------------------------------------")
print_metrics(train['fare_amount'],y_train_pred,independent_variables)
print("--------------------------------------")
print("Evaluation metrics for test data")
print("--------------------------------------")
print_metrics(test['fare_amount'], y_test_predicted,independent_variables)


Evaluation metrics for training data
--------------------------------------
RMSE is 1.2988634355847761
--------------------------------------
Evaluation metrics for test data
--------------------------------------
RMSE is 5.623981075488452


In [36]:
from sklearn.model_selection import GridSearchCV
estimator = RandomForestRegressor()
param_grid = {        
            "max_features"      : ["auto", "sqrt", "log2"],
            "min_samples_split" : [2,4,5,6],
            "bootstrap": [True, False],
            }

grid = GridSearchCV(estimator, param_grid, n_jobs=-1, cv=5,verbose=5)

grid.fit(train[independent_variables],train['fare_amount'])

Fitting 5 folds for each of 18 candidates, totalling 90 fits


Error: Canceled future for execute_request message before replies were done

In [35]:
y_test_predicted = grid.predict(test[independent_variables])
y_train_pred = grid.predict(train[independent_variables])
print("Evaluation metrics for training data")
print("--------------------------------------")
print_metrics(train['fare_amount'],y_train_pred,independent_variables)
print("--------------------------------------")
print("Evaluation metrics for test data")
print("--------------------------------------")
print_metrics(test['fare_amount'], y_test_predicted,independent_variables)

Evaluation metrics for training data
--------------------------------------
RMSE is 2.032773972928882
--------------------------------------
Evaluation metrics for test data
--------------------------------------
RMSE is 5.649653294470607
